# Comparing GO and KM results

In [275]:
#imports
import os
import csv

In [276]:
#filepaths
GO_dir = "/Users/student/Documents/Wi20-SysBio/GSEA"
metagene_dir = "/Users/student/Documents/Wi20-SysBio/metagenes/mda"

In [323]:
paths = get_paths(GO_dir, "MDA")

In [324]:
paths

{'1_vs_proxy_parental_UP': '/Users/student/Documents/Wi20-SysBio/GSEA/MDA/1_vs_proxy_parental_UP_prerank_report_GO/gseapy.prerank.gene_sets.report.csv',
 '2_vs_proxy_parental_UP': '/Users/student/Documents/Wi20-SysBio/GSEA/MDA/2_vs_proxy_parental_UP_prerank_report_GO/gseapy.prerank.gene_sets.report.csv',
 '3_vs_proxy_parental_DOWN': '/Users/student/Documents/Wi20-SysBio/GSEA/MDA/3_vs_proxy_parental_DOWN_prerank_report_GO/gseapy.prerank.gene_sets.report.csv',
 '4_vs_proxy_parental_UP': '/Users/student/Documents/Wi20-SysBio/GSEA/MDA/4_vs_proxy_parental_UP_prerank_report_GO/gseapy.prerank.gene_sets.report.csv',
 '7_vs_proxy_parental_DOWN': '/Users/student/Documents/Wi20-SysBio/GSEA/MDA/7_vs_proxy_parental_DOWN_prerank_report_GO/gseapy.prerank.gene_sets.report.csv',
 '7_vs_proxy_parental_UP': '/Users/student/Documents/Wi20-SysBio/GSEA/MDA/7_vs_proxy_parental_UP_prerank_report_GO/gseapy.prerank.gene_sets.report.csv',
 '13_vs_proxy_parental_UP': '/Users/student/Documents/Wi20-SysBio/GSEA/MDA

In [326]:
fm = filter_metagenes(paths, 0.05, 0.05,  f'MDA_GO_filtered.csv')

In [331]:
sd = survival_data("/Users/student/Documents/Wi20-SysBio/GSEA/MDA_filtered.csv", 0.5, 0)

In [332]:
sd

{'MDA_1_vs_proxy_parental_UP_FC_0': (3.2e-05, 0.659271571, 400, 1580),
 'MDA_7_vs_proxy_parental_DOWN_FC_0': (7.69e-05, 0.654868116, 324, 1656),
 'MDA_2_vs_proxy_parental_UP_FC_0': (0.000181291, 0.654530454, 275, 1705),
 'MDA_3_vs_proxy_parental_UP_FC_0': (5.1e-05, 0.650045483, 329, 1651),
 'MDA_10_vs_proxy_parental_DOWN_FC_0': (5.29e-07, 0.645081151, 794, 1186),
 'MDA_6_vs_proxy_parental_UP_FC_0': (6.16e-07, 0.643237742, 683, 1297),
 'MDA_Met-Like_parental_vs_rest_UP_FC_0': (2.77e-06, 0.628574694, 397, 1583),
 'MDA_12_vs_proxy_parental_DOWN_FC_0': (3.77e-08, 0.617058009, 711, 1269),
 'MDA_5_vs_proxy_parental_UP_FC_0': (1.79e-08, 0.61168646, 761, 1219),
 'MDA_13_vs_proxy_parental_UP_FC_0': (2.58e-05, 0.611308754, 243, 1737),
 'MDA_p-Low_vs_p-BoM_UP_FC_0': (3.2e-06, 0.610110954, 314, 1666),
 'MDA_10_vs_proxy_parental_UP_FC_0': (5.56e-06, 0.601406485, 265, 1715),
 'MDA_3_vs_proxy_parental_DOWN_FC_0': (6.37e-09, 0.600370431, 925, 1055),
 'MDA_p-Low_vs_p-Brm2_DOWN_FC_0': (1.81e-05, 0.59386

In [278]:
#survival_analysis = "/Users/student/Documents/Wi20-SysBio/GSEA/HCC_filtered.csv"
survival_analysis = "/Users/student/Documents/Wi20-SysBio/GSEA/MDA_filtered.csv"

In [279]:
#dictionary of csv paths
def get_paths(GO_dir, batch):
    csv_paths = {}
    batch_dir = os.path.join(GO_dir, batch)
    for f in os.listdir(batch_dir):
        #get all directories
        cluster_dir = os.path.join(batch_dir, f)
        if os.path.isdir(cluster_dir):
            cluster_dir_name = f.split('_prerank')[0]
            for g in os.listdir(cluster_dir):
                if g[-4:] == '.csv': #found csv
                    csv_paths[cluster_dir_name] = os.path.join(cluster_dir, g)
    return csv_paths

In [282]:
paths = get_paths(GO_dir, batch)

In [ ]:
fm = filter_metagenes(paths, 0.05, 0.05,  f'{batch}_GO_filtered.csv')

In [325]:
def filter_metagenes(csv_paths, pval_cutoff, fdr_cutoff, filename):
    diff_exp = {}
    dex = []
    
    csv_columns = None
    for comp, csv_f in csv_paths.items():
        diff_exp[comp] = []
        with open(csv_f) as f:
            input_file = csv.DictReader(f)
            for gene_set in input_file:
                if csv_columns is None:
                    csv_columns = ["pair"] + list(gene_set.keys()) 
                try:
                    if float(gene_set['pval']) < pval_cutoff and float(gene_set['es'])*float(gene_set['fdr']) < fdr_cutoff:
                        diff_exp[comp].append(gene_set) 
                        
                        gene_set['pair'] = comp
                        dex.append(gene_set)
                #ignore results that are missing a p-val 
                except:
                    pass
        diff_exp[comp] = sorted(diff_exp[comp], key = lambda x: float(x['es']), reverse = True) #sort by enrichment
    
    #write filtered to csv
    with open(os.path.join(GO_dir, filename), 'w', newline="") as f:  
        writer = csv.DictWriter(f, fieldnames=csv_columns)
        writer.writeheader()
        for data in dex:
            writer.writerow(data)
    
    return diff_exp
    

In [310]:
# read in KM data, filter on KM
def survival_data(survival_analysis, HZ_cutoff, FC_cutoff):
    survival_data = {}
    with open(survival_analysis) as f:
        input_file = csv.DictReader(f)
        for KM in input_file:
            if float(KM['HZ_KM']) > HZ_cutoff and KM['signature'][-1] == str(FC_cutoff): #only consider fold change threshold 1 for now
                survival_data[KM['signature']] = (float(KM['pval_KM']), float(KM['HZ_KM']), int(KM['n_low']), int(KM['n_high']))
    return survival_data

In [257]:
#read in metagenes, enforce fc 2 cutoff
def read_metagenes(metagene_dir, FC_cutoff ):
    metagene_dict = {}
    
    for subdir in ["/UP", "/DOWN"]:
        for file in os.listdir(metagene_dir + subdir):
            metagene_dict[file.replace(".csv", "")] = []
            with open(os.path.join(metagene_dir + subdir, file)) as f:
                input_file = csv.DictReader(f)
                for gene in input_file:
                    if float(gene['logfoldchange']) > FC_cutoff:
                        metagene_dict[file.replace(".csv", "")].append(gene['genes'])

    return metagene_dict

In [297]:
# for each survival analysis that looks interesting, look at overlap in GO enriched vs our metagenes
def calc_overlap(filename, survival_data, diff_exp, metagene_dict):
    results = []
    for KM in survival_data:
        key = '_'.join(KM.split('_')[1:-2])
        print(KM, key)
        #print(survival_data[KM])
        if key not in diff_exp:
            continue
        for gene_group in diff_exp[key]:
            gene_list = set(gene_group['ledge_genes'].split(';'))
            metagene = metagene_dict[key]
            #print(gene_list, metagene)
            overlap = [mg for mg in metagene if mg in gene_list]
            result_dict = {'key':key, 'term':gene_group['Term'], 'es':gene_group['es'], 'overlap':";".join(overlap), 
                           'ledge':gene_group['ledge_genes'], 'metagene':";".join([mg for mg in metagene]), 
                            'perc': len(overlap) / float(len(gene_list))}
            results.append(result_dict)
            #print(key, gene_group['Term'], gene_group['es'], overlap, overlap / float(len(gene_list)))
            
    csv_columns = ['key', 'term', 'es', 'overlap', 'ledge', 'metagene', 'perc']
    
    #write filtered to csv
    with open(os.path.join(GO_dir, filename), 'w', newline="") as f:  
        writer = csv.DictWriter(f, fieldnames=csv_columns)
        writer.writeheader()
        for result_dict in results:
            writer.writerow(result_dict)

In [335]:
def run_all(GO_dir, batch, metagene_dir, survival_analysis, pval_cutoff, fdr_cutoff, hz_cutoff, fc_cutoff):
    csv_paths = get_paths(GO_dir, batch)
    diff_exp = filter_metagenes(csv_paths, pval_cutoff, fdr_cutoff, f'{batch}_GO_filtered.csv')
    sa = survival_data(survival_analysis, hz_cutoff, fc_cutoff)
    metagene_dict = read_metagenes(metagene_dir, fc_cutoff)
    calc_overlap(f"{batch}_overlap_FC-{fc_cutoff}_HZ-{hz_cutoff}.csv", sa, diff_exp, metagene_dict)

In [336]:
#filepaths
GO_dir = "/Users/student/Documents/Wi20-SysBio/GSEA"
filepaths = [["MDA", "/Users/student/Documents/Wi20-SysBio/metagenes/mda", "/Users/student/Documents/Wi20-SysBio/GSEA/MDA_filtered.csv"],
             ["HCC_MK", "/Users/student/Documents/Wi20-SysBio/metagenes/mk",  "/Users/student/Documents/Wi20-SysBio/GSEA/HCC_filtered.csv"]]

pval_cutoff, fdr_cutoff = 0.05, 0.05 

fc_cutoffs = [0, 1, 2]
hz_cutoffs = [0.5, 0.7, 1]
for batch_info in filepaths:
    for fc_cutoff in fc_cutoffs:
        for hz_cutoff in hz_cutoffs:
            print(batch_info, fc_cutoff, hz_cutoffs)
            run_all(GO_dir, *batch_info, pval_cutoff, fdr_cutoff, hz_cutoff, fc_cutoff)

['MDA', '/Users/student/Documents/Wi20-SysBio/metagenes/mda', '/Users/student/Documents/Wi20-SysBio/GSEA/MDA_filtered.csv'] 0 [0.5, 0.7, 1]
MDA_1_vs_proxy_parental_UP_FC_0 1_vs_proxy_parental_UP
MDA_7_vs_proxy_parental_DOWN_FC_0 7_vs_proxy_parental_DOWN
MDA_2_vs_proxy_parental_UP_FC_0 2_vs_proxy_parental_UP
MDA_3_vs_proxy_parental_UP_FC_0 3_vs_proxy_parental_UP
MDA_10_vs_proxy_parental_DOWN_FC_0 10_vs_proxy_parental_DOWN
MDA_6_vs_proxy_parental_UP_FC_0 6_vs_proxy_parental_UP
MDA_Met-Like_parental_vs_rest_UP_FC_0 Met-Like_parental_vs_rest_UP
MDA_12_vs_proxy_parental_DOWN_FC_0 12_vs_proxy_parental_DOWN
MDA_5_vs_proxy_parental_UP_FC_0 5_vs_proxy_parental_UP
MDA_13_vs_proxy_parental_UP_FC_0 13_vs_proxy_parental_UP
MDA_p-Low_vs_p-BoM_UP_FC_0 p-Low_vs_p-BoM_UP
MDA_10_vs_proxy_parental_UP_FC_0 10_vs_proxy_parental_UP
MDA_3_vs_proxy_parental_DOWN_FC_0 3_vs_proxy_parental_DOWN
MDA_p-Low_vs_p-Brm2_DOWN_FC_0 p-Low_vs_p-Brm2_DOWN
MDA_9_vs_proxy_parental_DOWN_FC_0 9_vs_proxy_parental_DOWN
MDA_6_vs_

HCC_8_vs_p-L_DOWN_FC_2 8_vs_p-L_DOWN
HCC_LM-H_vs_p-L_UP_FC_2 LM-H_vs_p-L_UP
HCC_8_vs_p-L_UP_FC_2 8_vs_p-L_UP
HCC_LM-H_vs_LM-L_UP_FC_2 LM-H_vs_LM-L_UP
['HCC_MK', '/Users/student/Documents/Wi20-SysBio/metagenes/mk', '/Users/student/Documents/Wi20-SysBio/GSEA/HCC_filtered.csv'] 2 [0.5, 0.7, 1]
HCC_8_vs_p-L_DOWN_FC_2 8_vs_p-L_DOWN
